In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["MKL_NUM_THREADS"] = "8" 
os.environ["NUMEXPR_NUM_THREADS"] = "8" 
os.environ["OMP_NUM_THREADS"] = "8" 

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc
import wandb 
from fastxtend.vision.all import EMACallback


In [3]:
!ls ../data/

Ribonanza_bpp_files	  split
all_test_data.npy	  supplementary_silico_predictions
eterna_openknot_metadata  test_sequences.csv
rcentral.parquet	  train_corrected.parquet
sample_submission.csv	  train_data.csv
sequence_libraries	  train_data.parquet


In [3]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 4
    device = 'cuda'
    seed = 2023
    out = 'exp_32_v2'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV6SAVED'
    sn_train = False
    
    model_name = 'RNA_ModelV25Corrected'
    model_kwargs = dict(dim=192 * 2,
        depth=4,
        head_size=32,
        drop_pat_dropout=0.2,
        dropout=0.2,
        bpp_transfomer_depth = 4)

   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
    wandb = True
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

def class_to_dict(cls):
    # Create a dictionary from the class attributes
    return {key: value for key, value in cls.__dict__.items() if not key.startswith("__") and not callable(value)}

In [4]:
split = pd.read_csv(CFG.split_id)
df = pd.read_parquet(CFG.path/'train_corrected.parquet')
df = pd.merge(df, split, on='sequence_id')
#df = df.query("SN_filter==1").reset_index(drop=True)
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)


ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train',sn_train=CFG.sn_train)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_size=CFG.bs, num_workers=CFG.num_workers,shuffle=True,
            persistent_workers=True), CFG.device)


dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_size=CFG.bs * 2, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
del split
del df
gc.collect()


59

In [ ]:

learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)(**CFG.model_kwargs).cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    #WandbCallback(log_preds=False),
                    CSVLogger(),
                     EMACallback(),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)
wandb.finish()

epoch,train_loss,valid_loss,mae,time
0,0.126599,0.214973,0.168818,39:28


Better model found at epoch 0 with mae value: 0.16881751441260928.


In [ ]:
res = dict()
for l in df_valid["L"].unique():
    ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval')
    ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval', mask_only=True)
    sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
    len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
                   drop_last=False)
    dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
                   batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

    data = DataLoaders(dl_val,dl_val)
    learn = Learner(data,
                    getattr(rnacomp.models, CFG.model_name)(dim = CFG.dim, graph_layers=CFG.graph_layers).cuda(), 
                    path = CFG.out, 
                    loss_func=loss_laplace,
                    metrics=[MAE()]).to_fp16() 
    learn.load('model')
    learn.eval()
    loss_, score_ = learn.validate()
    res[l]= score_

res = pd.DataFrame(pd.Series(res)).reset_index()
res.columns = ['L', 'mae']
res.sort_values(by="L")

In [ ]:
res

In [ ]:
res